In [ ]:
#Import packages
import pandas as pd
import requests
import json

from config import gkey

In [ ]:
#load crash data
file_path = 'Resources/CRASH DATA SET'

crash_csv = pd.read_csv(file_path)

place_holder_df = pd.DataFrame()

In [ ]:
#set list of lat lng to feed into google

lat = df['Start_lat']
lng = df['Start_lng']

for row in crash_csv:
    #lat = df['Start_lat']
    #lng = df['Start_lng']
    google_path = f'{lat},{lng}'
    params = {'Key': gkey,
         'Path' : google_path}
    base_url = 'https://roads.googleapis.com/v1/speedLimits?'

    response = requests.get(base_url, params=params)
    
    roads_data = response.json()
    
    
    

In [ ]:
# convert to json
roads_data = response.json()

